In [ ]:
# ============================================
# 🔧 SETUP
# ============================================
!pip install -q pydub torchaudio transformers scikit-learn
!apt-get -y install ffmpeg  # for pydub to handle .m4a
!pip install datasets peft accelerate
!pip install speechbrain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remov

In [ ]:
# Install dependencies
!pip install speechbrain torchaudio mir_eval pesq

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.8/102.8 kB 6.7 MB/s eta 0:00:00
  Created wheel for pesq: filename=pesq-0.0.4-cp311-cp311-linux_x86_64.whl size=274953 sha256=e7eb849da8650473e635ce2d8e04cdd762f62c202f27eebf294177ffbf18d101
  Stored in directory: /root/.cache/pip/wheels/ae/f1/23/2698d0bf31eec2b2aa50623b5d93b6206c49c7155d0e31345d
Successfully built pesq


In [ ]:
import os
import numpy as np
import torch
from pydub import AudioSegment
from sklearn.metrics import roc_curve
from torch.nn.functional import cosine_similarity
from transformers import AutoModel, AutoProcessor
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import Dataset
from pathlib import Path
import random
from transformers import Wav2Vec2Model, Wav2Vec2Processor
from peft import get_peft_model, LoraConfig, TaskType, PeftModel
import torch.nn as nn
import torch.nn.functional as F
import math
import glob
import torchaudio
import soundfile as sf
import pandas as pd
from scipy.spatial.distance import cosine
from sklearn.preprocessing import normalize as np_normalize
from pesq import pesq
from mir_eval.separation import bss_eval_sources
from speechbrain.inference import SepformerSeparation as separator
from scipy.io.wavfile import write
import csv

In [ ]:
# ============================================
# 💾 Load Hugging Face Wav2Vec2 Model
# ============================================
model_name = "facebook/wav2vec2-base"
device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModel.from_pretrained(model_name).to(device)
processor = AutoProcessor.from_pretrained(model_name) #tokenizer

print(f"Using device: {device}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:315: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/380M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:315: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Using device: cuda


In [ ]:
# ============================================
# 🔊 Load .m4a as tensor
# ============================================
def load_m4a_for_model(file_path, sr=16000):
    """
    Load an .m4a file, convert to mono & 16kHz, and return as a PyTorch tensor.
    """
    try:
        audio = AudioSegment.from_file(file_path, format="m4a").set_channels(1).set_frame_rate(sr)
        samples = np.array(audio.get_array_of_samples()).astype(np.float32) / 32768.0
        tensor = torch.tensor(samples).unsqueeze(0)  # (1, T)
        return tensor, sr
    except Exception as e:
        print(f"Error loading or processing {file_path}: {e}")
        return None, None

In [ ]:
def evaluate(trial_list_path, base_dir, pooling_strategy="mean"):
    scores, labels = [], []
    line_count = 0
    max_lines = 500

    with open(trial_list_path, "r") as f:
        for line in f:
            if line_count >= max_lines:
                break  # Exit the loop after reading the first max_lines

            parts = line.strip().split()
            if len(parts) != 3:
                print(f"Skipping invalid line: {line.strip()}")
                continue

            label, path1, path2 = parts
            full_path1 = os.path.join(base_dir, path1)
            full_path2 = os.path.join(base_dir, path2)
            try:
                label = int(label)
            except ValueError:
                print(f"Invalid label '{label}' in line: {line.strip()}")
                continue

            emb1 = get_embedding(full_path1, pooling_strategy=pooling_strategy)
            emb2 = get_embedding(full_path2, pooling_strategy=pooling_strategy)

            if emb1 is not None and emb2 is not None:
                try:
                    sim = 1 - cosine(emb1, emb2) # Cosine similarity
                    scores.append(sim)
                    labels.append(label)
                except Exception as e:
                    print(f"❌ Error computing cosine similarity for {path1}, {path2}: {e}")
            else:
                print(f"❌ Could not get embeddings for one or both files: {path1}, {path2}")

            line_count += 1
            # if line_count % 50 == 0:  # Optional: Print progress
            #     print(f"Processed {line_count}/{max_lines} trial pairs...")

    return np.array(scores), np.array(labels)

In [ ]:
# ============================================
# 🧪 Evaluate speaker verification
# ============================================
def evaluate(trial_list_path, base_dir):
    scores, labels = [], []
    line_count = 0
    max_lines = 1000

    with open(trial_list_path, "r") as f:
        for line in f:
            if line_count >= max_lines:
                break  # Exit the loop after reading the first max_lines

            parts = line.strip().split()
            if len(parts) != 3:
                print(f"Skipping invalid line: {line.strip()}")
                continue

            label, path1, path2 = parts
            full_path1 = os.path.join(base_dir, path1)
            full_path2 = os.path.join(base_dir, path2)
            try:
                label = int(label)
            except ValueError:
                print(f"Invalid label '{label}' in line: {line.strip()}")
                continue

            emb1 = get_embedding(full_path1, pooling_strategy=pooling_strategy)
            emb2 = get_embedding(full_path2, pooling_strategy=pooling_strategy)

            if emb1 is not None and emb2 is not None:
                try:
                    sim = 1 - cosine(emb1, emb2) # Cosine similarity
                    scores.append(sim)
                    labels.append(label)
                except Exception as e:
                    print(f"Error computing cosine similarity for {path1}, {path2}: {e}")
            else:
                print(f"Could not get embeddings for one or both files: {path1}, {path2}")

            line_count += 1
            if line_count % 50 == 0:  # Optional: Print progress
                print(f"Processed {line_count}/{max_lines} trial pairs...")

    return np.array(scores), np.array(labels)


In [ ]:
# ============================================
# 📊 Compute EER and TAR@1%FAR
# ============================================
def compute_metrics(scores, labels):
    fpr, tpr, thresholds = roc_curve(labels, scores)
    eer_idx = np.nanargmin(np.abs(fpr - (1 - tpr)))
    eer = (fpr[eer_idx] + (1 - tpr[eer_idx])) / 2 * 100

    target_far = 0.01
    tar_idx = np.argmin(np.abs(fpr - target_far))
    tar_at_1far = tpr[tar_idx] * 100

    return eer, tar_at_1far

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ============================================
# 🚀 Run Evaluation
# ============================================

base_vox_path = "/content/drive/MyDrive/SU_P2/vox1/vox1_test_wav/wav"
trial_file = "/content/drive/MyDrive/SU_P2/vox1/vox1_test_wav/vox1(cleaned).txt"

print("🧪 Evaluating Speaker Verification on VoxCeleb1 (m4a)...")
scores, labels = evaluate(trial_file, base_vox_path)
eer, tar = compute_metrics(scores, labels)

print("\n✅ Evaluation Results:")
print(f"  Equal Error Rate (EER): {eer:.2f}%")
print(f"  TAR @ 1% FAR: {tar:.2f}%")



🧪 Evaluating Speaker Verification on VoxCeleb1 (m4a)...
Processed 50/500 trial pairs...
Processed 100/500 trial pairs...
Processed 150/500 trial pairs...
Processed 200/500 trial pairs...
Processed 250/500 trial pairs...
Processed 300/500 trial pairs...
Processed 350/500 trial pairs...
Processed 400/500 trial pairs...
Processed 450/500 trial pairs...
Processed 500/500 trial pairs...

✅ Evaluation Results:
  Equal Error Rate (EER): 32.80%
  TAR @ 1% FAR: 10.80%


# **Finetuning**

 Part 1: Dataset Preparation (VoxCeleb2)

In [ ]:
class VoxCeleb2Dataset(Dataset):
    def __init__(self, base_path, speaker_ids, sample_rate=16000, max_duration=3.0):
        self.base_path = Path(base_path)
        self.speaker_ids = speaker_ids
        self.sample_rate = sample_rate
        self.max_len = int(sample_rate * max_duration)  # e.g., 3 seconds
        self.data = []
        self.label_map = {spk: i for i, spk in enumerate(sorted(speaker_ids))}

        for spk in speaker_ids:
            spk_path = self.base_path / spk
            files = list(spk_path.glob("*.m4a"))
            for f in files:
                self.data.append((f, self.label_map[spk]))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        file_path, label = self.data[idx]
        waveform = self.load_m4a(file_path)

        # Pad/Trim
        if len(waveform) > self.max_len:
            waveform = waveform[:self.max_len]
        else:
            pad_len = self.max_len - len(waveform)
            waveform = np.pad(waveform, (0, pad_len))

        return torch.tensor(waveform, dtype=torch.float32), label

    def load_m4a(self, path):
        audio = AudioSegment.from_file(path, format="m4a").set_channels(1).set_frame_rate(self.sample_rate)
        samples = np.array(audio.get_array_of_samples()).astype(np.float32) / 32768.0
        return samples


In [ ]:
# Paths
vox2_path = "/content/drive/MyDrive/SU_P2/aac1"

# Get sorted speaker IDs
speaker_dirs = sorted([d.name for d in Path(vox2_path).iterdir() if d.is_dir()])
train_ids = speaker_dirs[:100]
test_ids = speaker_dirs[100:118]

# Create dataset
train_dataset = VoxCeleb2Dataset(vox2_path, train_ids)
test_dataset = VoxCeleb2Dataset(vox2_path, test_ids)

# Optional: Dataloader
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)


In [ ]:
class ArcFace(nn.Module):
    def __init__(self, in_features, out_features, s=30.0, m=0.50):
        super().__init__()
        self.fc = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.fc)
        self.s = s  # scale
        self.m = m  # margin

    def forward(self, x, labels):
        # Normalize
        x = F.normalize(x, p=2, dim=1)
        w = F.normalize(self.fc, p=2, dim=1)

        # Cosine similarity
        cos_theta = torch.matmul(x, w.T)
        theta = torch.acos(torch.clamp(cos_theta, -1.0 + 1e-7, 1.0 - 1e-7))

        # Add margin to target class
        one_hot = F.one_hot(labels, num_classes=w.size(0)).float().to(x.device)
        logits = torch.cos(theta + self.m) * one_hot + cos_theta * (1.0 - one_hot)

        return self.s * logits


In [ ]:
class Wav2Vec2SpeakerVerification(nn.Module):
    def __init__(self, pretrained_model="facebook/wav2vec2-base", num_classes=100, use_lora=True):
        super().__init__()

        # Load pretrained model
        base_model = Wav2Vec2Model.from_pretrained(pretrained_model)

        # Freeze base model
        for param in base_model.parameters():
            param.requires_grad = False

        base_model.gradient_checkpointing_disable()

        if use_lora:
            # Apply LoRA to projection layers
            lora_config = LoraConfig(
                r=8,
                lora_alpha=16,
                target_modules=["out_proj"],
                lora_dropout=0.1,
                bias="none",
                task_type=TaskType.FEATURE_EXTRACTION,
            )
            self.encoder = get_peft_model(base_model, lora_config)
        else:
            self.encoder = base_model

        self.embedding_dim = base_model.config.hidden_size
        self.arcface = ArcFace(self.embedding_dim, num_classes)

    def forward(self, audio, labels=None):
        # Forward through base model (with or without LoRA)
        if isinstance(self.encoder, PeftModel):
            outputs = self.encoder.base_model(input_values=audio)
        else:
            outputs = self.encoder(input_values=audio)

        # Mean pool over time (can replace with custom pooling)
        pooled = outputs.last_hidden_state.mean(dim=1)

        if labels is not None:
            return self.arcface(pooled, labels.to(audio.device))
        return pooled

In [ ]:
#If GPU is available
model = Wav2Vec2SpeakerVerification(num_classes=100).to("cuda")

#cpu
# model = Wav2Vec2SpeakerVerification(num_classes=100)

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:315: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [ ]:
def train(model, train_loader, epochs=10, lr=1e-4, device="cuda"):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr)

    for epoch in range(epochs):
        print(f"\n🚀 Epoch {epoch + 1}/{epochs}")
        model.train()
        train_loss, correct, total = 0.0, 0, 0

        for audio, labels in tqdm(train_loader, desc="🏋️ Training", leave=False):
            audio, labels = audio.to(device), labels.to(device)

            optimizer.zero_grad()
            logits = model(audio, labels)  # ArcFace returns logits
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * labels.size(0)
            preds = logits.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = correct / total
        train_loss /= total
        print(f"✅ Train Loss: {train_loss:.4f} | Train Accuracy: {train_acc:.4f}")

    # Save final model
    torch.save(model.state_dict(), "final_speaker_model.pth")
    print("💾 Final model saved")
    print("\n🎉 Training complete!")


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
train(model, train_loader, epochs=10, lr=1e-4, device=device)#cuda if available


🚀 Epoch 1/10


🏋️ Training:  47%|████▋     | 878/1865 [1:19:04<1:17:41,  4.72s/it]

In [ ]:
print(f"Number of samples in training dataset: {len(train_dataset)}")
print(f"Batch size: {train_loader.batch_size}")
print(f"Expected iterations per epoch: {len(train_loader)}")


# **Part 3**

 Step 1: Improve the script to mix multiple utterances per speaker pair

In [ ]:
# Paths
vox_root = "/content/drive/MyDrive/programmingAssignmentShreya/aac1"
mixed_train_dir = "/content/drive/MyDrive/programmingAssignmentShreya/mixed_train_audio"
mixed_test_dir = "/content/drive/MyDrive/programmingAssignmentShreya/mixed_test_audio"
ref_dir = "/content/drive/MyDrive/programmingAssignmentShreya/clean_refs"
metadata_csv = "/content/drive/MyDrive/programmingAssignmentShreya/mixed_metadata.csv"

os.makedirs(mixed_train_dir, exist_ok=True)
os.makedirs(mixed_test_dir, exist_ok=True)
os.makedirs(ref_dir, exist_ok=True)

# Get speaker folders and split
all_speakers = sorted([d for d in os.listdir(vox_root) if d.startswith("id")])
train_speakers = all_speakers[:50]
test_speakers = all_speakers[50:100]

def get_random_utterance(speaker_id):
    files = glob.glob(os.path.join(vox_root, speaker_id, "*.m4a"))
    if not files:
        return None
    return random.choice(files)

def load_and_resample(path, target_sr=16000):
    waveform, sr = torchaudio.load(path)
    if sr != target_sr:
        waveform = torchaudio.transforms.Resample(orig_freq=sr, new_freq=target_sr)(waveform)
    return waveform.mean(dim=0).unsqueeze(0)  # Mono

def mix_pair(speaker1, speaker2, out_dir, mix_id):
    utt1 = get_random_utterance(speaker1)
    utt2 = get_random_utterance(speaker2)
    if utt1 is None or utt2 is None:
        return False

    wav1 = load_and_resample(utt1)
    wav2 = load_and_resample(utt2)

    min_len = min(wav1.shape[1], wav2.shape[1])
    wav1 = wav1[:, :min_len]
    wav2 = wav2[:, :min_len]
    mix = wav1 + wav2

    mix_path = os.path.join(out_dir, f"mix{mix_id}.wav")
    spk1_path = os.path.join(ref_dir, f"mix{mix_id}_spk1_clean.wav")
    spk2_path = os.path.join(ref_dir, f"mix{mix_id}_spk2_clean.wav")

    sf.write(mix_path, mix.squeeze(0).numpy(), 16000)
    sf.write(spk1_path, wav1.squeeze(0).numpy(), 16000)
    sf.write(spk2_path, wav2.squeeze(0).numpy(), 16000)

    with open(metadata_csv, "a") as f:
        f.write(f"mix{mix_id}, {mix_path}, {utt1}, {utt2}, {spk1_path}, {spk2_path}\n")

    return True

# Generate mixtures for all unique speaker pairs in training set
print("\n🔁 Generating training mixtures...")
mix_id = 1
for i in range(len(train_speakers)):
    for j in range(i + 1, len(train_speakers)):
        mix_pair(train_speakers[i], train_speakers[j], mixed_train_dir, mix_id)
        mix_id += 1

# Generate mixtures for all unique speaker pairs in testing set
print("\n🔁 Generating testing mixtures...")
for i in range(len(test_speakers)):
    for j in range(i + 1, len(test_speakers)):
        mix_pair(test_speakers[i], test_speakers[j], mixed_test_dir, mix_id)
        mix_id += 1

print("\n✅ Done generating mixtures and references!")



🔁 Generating training mixtures...

🔁 Generating testing mixtures...

✅ Done generating mixtures and references!


In [ ]:
# Set paths
#MIXED_DIR = "/content/drive/MyDrive/programmingAssignmentShreya/mixed_test_audio"
MIXED_DIR="/content/drive/MyDrive/SU_P2/mixed_test_trial_audio"
CLEAN_DIR = "/content/drive/MyDrive/SU_P2/clean_refs"
OUTPUT_CSV = "/content/drive/MyDrive/SU_P2/separation_evaluation_results_trial.csv"


In [ ]:

# Initialize SepFormer
sepformer = separator.from_hparams(source="speechbrain/sepformer-whamr", savedir="tmpdir_sepformer")

# Function to load and resample audio
def load_audio(path, target_sr=16000):
    waveform, sr = torchaudio.load(path)
    if sr != target_sr:
        transform = torchaudio.transforms.Resample(orig_freq=sr, new_freq=target_sr)
        waveform = transform(waveform)
    return waveform.squeeze().numpy()

# Prepare results
results = []

# Process each mixed file
for file in os.listdir(MIXED_DIR):
    if not file.endswith(".wav"):
        continue

    base_name = os.path.splitext(file)[0]
    mix_path = os.path.join(MIXED_DIR, file)
    clean1_path = os.path.join(CLEAN_DIR, f"{base_name}_spk1_clean.wav")
    clean2_path = os.path.join(CLEAN_DIR, f"{base_name}_spk2_clean.wav")

    if not (os.path.exists(clean1_path) and os.path.exists(clean2_path)):
        print(f"Missing clean files for: {base_name}")
        continue

    print(f"Processing: {file}")

    # Load audio
    mix = load_audio(mix_path)
    ref1 = load_audio(clean1_path)
    ref2 = load_audio(clean2_path)

    # Pad/trim clean signals to match
    min_len = min(len(ref1), len(ref2), len(mix))
    ref1, ref2, mix = ref1[:min_len], ref2[:min_len], mix[:min_len]

    # Separate
    est_sources = sepformer.separate_file(path=mix_path).numpy()

    # Access the two sources from the third dimension
    est1 = est_sources[0, :, 0]  # Source 1 from the third dimension
    est2 = est_sources[0, :, 1]  # Source 2 from the third dimension

    # Pad/trim estimated sources to match references
    est1 = np.pad(est1, (0, max(0, len(ref1) - len(est1))), 'constant')[:len(ref1)]
    est2 = np.pad(est2, (0, max(0, len(ref2) - len(est2))), 'constant')[:len(ref2)]

    # Create arrays for BSS eval
    refs = np.vstack([ref1, ref2])
    ests = np.vstack([est1, est2])

    # Align sources for optimal permutation
    sdr, sir, sar, perm = bss_eval_sources(refs, ests)

    # Apply best permutation
    est1_aligned = ests[perm[0]]
    est2_aligned = ests[perm[1]]
    ref1_aligned = refs[0]
    ref2_aligned = refs[1]

    # Compute PESQ (narrow-band mode)
    try:
        pesq1 = pesq(16000, ref1_aligned, est1_aligned, 'nb')
        pesq2 = pesq(16000, ref2_aligned, est2_aligned, 'nb')
    except Exception as e:
        print(f"PESQ error on {file}: {e}")
        pesq1, pesq2 = None, None

    # Log result
    results.append({
        "file": base_name,
        "SDR_spk1": sdr[perm[0]],
        "SIR_spk1": sir[perm[0]],
        "SAR_spk1": sar[perm[0]],
        "PESQ_spk1": pesq1,
        "SDR_spk2": sdr[perm[1]],
        "SIR_spk2": sir[perm[1]],
        "SAR_spk2": sar[perm[1]],
        "PESQ_spk2": pesq2,
    })

# Save to CSV
df = pd.DataFrame(results)
df.to_csv(OUTPUT_CSV, index=False)
print(f"\n✅ Evaluation completed. Results saved to: {OUTPUT_CSV}")


Processing: mix2441.wav
Resampling the audio from 16000 Hz to 8000 Hz


<ipython-input-29-e98a2e8093b0>:56: FutureWarning: mir_eval.separation.bss_eval_sources
	Deprecated as of mir_eval version 0.8.
	It will be removed in mir_eval version 0.9.
  sdr, sir, sar, perm = bss_eval_sources(refs, ests)


Processing: mix2445.wav
Resampling the audio from 16000 Hz to 8000 Hz
Processing: mix2442.wav
Resampling the audio from 16000 Hz to 8000 Hz
Processing: mix2450.wav
Resampling the audio from 16000 Hz to 8000 Hz
Processing: mix2446.wav
Resampling the audio from 16000 Hz to 8000 Hz
Processing: mix2447.wav
Resampling the audio from 16000 Hz to 8000 Hz
Processing: mix2449.wav
Resampling the audio from 16000 Hz to 8000 Hz
Processing: mix2444.wav
Resampling the audio from 16000 Hz to 8000 Hz
Processing: mix2443.wav
Resampling the audio from 16000 Hz to 8000 Hz
Processing: mix2448.wav
Resampling the audio from 16000 Hz to 8000 Hz

✅ Evaluation completed. Results saved to: /content/drive/MyDrive/programmingAssignmentShreya/separation_evaluation_results_trial.csv


In [ ]:
# ==== CONFIG ====
MIXED_DIR="/content/drive/MyDrive/SU_P2/mixed_test_trial_audio"
CLEAN_DIR = "/content/drive/MyDrive/SU_P2/clean_refs"
SEPARATED_DIR = "/content/drive/MyDrive/SU_P2/separated_outputs"
METADATA_CSV = "/content/drive/MyDrive/SU_P2/separated_metadata_trial.csv"
os.makedirs(SEPARATED_DIR, exist_ok=True)

# ==== Load SepFormer ====
sepformer = separator.from_hparams(source="speechbrain/sepformer-whamr", savedir="tmpdir_sepformer")

# ==== Load Audio Helper ====
def load_audio(path, target_sr=16000):
    waveform, sr = torchaudio.load(path)
    if sr != target_sr:
        transform = torchaudio.transforms.Resample(orig_freq=sr, new_freq=target_sr)
        waveform = transform(waveform)
    return waveform.squeeze().numpy()

# ==== Track metadata ====
metadata = []

# ==== Process Each File ====
for file in os.listdir(MIXED_DIR):
    if not file.endswith(".wav"):
        continue

    base_name = os.path.splitext(file)[0]
    mix_path = os.path.join(MIXED_DIR, file)
    clean1_path = os.path.join(CLEAN_DIR, f"{base_name}_spk1_clean.wav")
    clean2_path = os.path.join(CLEAN_DIR, f"{base_name}_spk2_clean.wav")

    if not (os.path.exists(clean1_path) and os.path.exists(clean2_path)):
        print(f"Skipping: Missing clean files for {base_name}")
        continue

    print(f"🔄 Separating: {file}")

    # Load waveforms
    mix = load_audio(mix_path)
    ref1 = load_audio(clean1_path)
    ref2 = load_audio(clean2_path)

    # Match length
    min_len = min(len(mix), len(ref1), len(ref2))
    mix = mix[:min_len]
    ref1 = ref1[:min_len]
    ref2 = ref2[:min_len]

    # Separate
    est_sources = sepformer.separate_file(path=mix_path).numpy()
    est1 = est_sources[0, :, 0]
    est2 = est_sources[0, :, 1]

    # Pad/trim to match
    est1 = np.pad(est1, (0, max(0, min_len - len(est1))), 'constant')[:min_len]
    est2 = np.pad(est2, (0, max(0, min_len - len(est2))), 'constant')[:min_len]

    # Align permutation
    refs = np.vstack([ref1, ref2])
    ests = np.vstack([est1, est2])
    _, _, _, perm = bss_eval_sources(refs, ests)
    est1_aligned = ests[perm[0]]
    est2_aligned = ests[perm[1]]

    # Save separated audio
    sep1_name = f"{base_name}_pred_spk{perm[0]+1}.wav"
    sep2_name = f"{base_name}_pred_spk{perm[1]+1}.wav"
    sep1_path = os.path.join(SEPARATED_DIR, sep1_name)
    sep2_path = os.path.join(SEPARATED_DIR, sep2_name)
    write(sep1_path, 16000, est1_aligned.astype(np.float32))
    write(sep2_path, 16000, est2_aligned.astype(np.float32))

    # Log metadata
    metadata.append({
        "mixed_file": file,
        "separated_file_1": sep1_name,
        "separated_file_2": sep2_name
    })

# ==== Save Metadata CSV ====
df = pd.DataFrame(metadata)
df.to_csv(METADATA_CSV, index=False)
print(f"\n✅ All separated audio saved to: {SEPARATED_DIR}")
print(f"📄 Metadata CSV saved to: {METADATA_CSV}")


🔄 Separating: mix2441.wav
Resampling the audio from 16000 Hz to 8000 Hz


<ipython-input-30-6878745a706b>:69: FutureWarning: mir_eval.separation.bss_eval_sources
	Deprecated as of mir_eval version 0.8.
	It will be removed in mir_eval version 0.9.
  _, _, _, perm = bss_eval_sources(refs, ests)


🔄 Separating: mix2445.wav
Resampling the audio from 16000 Hz to 8000 Hz
🔄 Separating: mix2442.wav
Resampling the audio from 16000 Hz to 8000 Hz
🔄 Separating: mix2450.wav
Resampling the audio from 16000 Hz to 8000 Hz
🔄 Separating: mix2446.wav
Resampling the audio from 16000 Hz to 8000 Hz
🔄 Separating: mix2447.wav
Resampling the audio from 16000 Hz to 8000 Hz
🔄 Separating: mix2449.wav
Resampling the audio from 16000 Hz to 8000 Hz
🔄 Separating: mix2444.wav
Resampling the audio from 16000 Hz to 8000 Hz
🔄 Separating: mix2443.wav
Resampling the audio from 16000 Hz to 8000 Hz
🔄 Separating: mix2448.wav
Resampling the audio from 16000 Hz to 8000 Hz

✅ All separated audio saved to: /content/drive/MyDrive/programmingAssignmentShreya/separated_outputs
📄 Metadata CSV saved to: /content/drive/MyDrive/programmingAssignmentShreya/separated_metadata_trial.csv


# **Phase 2: Run Speaker Identification on Separated Audio**

In [ ]:
# ==== CONFIG ====
CLEAN_DIR = "/content/drive/MyDrive/SU_P2/clean_refs_trial"
SEP_DIR = "/content/drive/MyDrive/SU_P2/separated_outputs"
FINETUNED_MODEL_PATH = "/content/final_speaker_model.pth"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ==== Load Pretrained and Processor ====
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
base_model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base").to(DEVICE)
base_model.eval()

# ==== Load Fine-tuned Model (same arch) ====
finetuned_model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base").to(DEVICE)
state_dict = torch.load(FINETUNED_MODEL_PATH, map_location=DEVICE)
finetuned_model.load_state_dict(state_dict, strict=False)
finetuned_model.eval()

# ==== Load Audio ====
def load_audio(path, target_sr=16000):
    waveform, sr = torchaudio.load(path)
    if sr != target_sr:
        resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=target_sr)
        waveform = resampler(waveform)
    return waveform.squeeze()

# ==== Extract Embedding ====
def extract_embedding(audio_path, model):
    wav = load_audio(audio_path)
    inputs = processor(wav, sampling_rate=16000, return_tensors="pt", padding=True)
    with torch.no_grad():
        outputs = model(inputs.input_values.to(DEVICE)).last_hidden_state
        embedding = outputs.mean(dim=1).squeeze()  # Mean pooling
    return embedding.cpu()

# ==== Build Clean Speaker Embedding Bank ====
print("🔍 Building speaker embedding bank...")
speaker_bank_base = {}
speaker_bank_finetuned = {}

for fname in os.listdir(CLEAN_DIR):
    if "_clean.wav" not in fname:
        continue
    path = os.path.join(CLEAN_DIR, fname)
    speaker_id = fname.replace("_clean.wav", "")  # e.g., mix2450_spk1

    emb_base = extract_embedding(path, base_model)
    emb_finetuned = extract_embedding(path, finetuned_model)

    speaker_bank_base[speaker_id] = emb_base
    speaker_bank_finetuned[speaker_id] = emb_finetuned

print(f"✅ Speaker bank size: {len(speaker_bank_base)} speakers")

# ==== Identify Speakers from Separated Files ====
results = []

def get_predicted_speaker(query_emb, speaker_bank):
    scores = {spk: F.cosine_similarity(query_emb, emb, dim=0).item()
              for spk, emb in speaker_bank.items()}
    best_spk = max(scores, key=scores.get)
    return best_spk, scores[best_spk]

print("🎙️ Evaluating separated files...")

for fname in tqdm(os.listdir(SEP_DIR)):
    if not fname.endswith(".wav"):
        continue
    path = os.path.join(SEP_DIR, fname)
    true_speaker = fname.replace("_pred_spk1.wav", "").replace("_pred_spk2.wav", "")

    query_base = extract_embedding(path, base_model)
    query_finetuned = extract_embedding(path, finetuned_model)

    pred_base, sim_base = get_predicted_speaker(query_base, speaker_bank_base)
    pred_finetuned, sim_finetuned = get_predicted_speaker(query_finetuned, speaker_bank_finetuned)

    correct_base = int(pred_base == true_speaker)
    correct_finetuned = int(pred_finetuned == true_speaker)


    results.append({
        "file": fname,
        "true_speaker": true_speaker,
        "pred_base": pred_base,
        "pred_finetuned": pred_finetuned,
        "correct_base": correct_base,
        "correct_finetuned": correct_finetuned,
        "sim_base": sim_base,
        "sim_finetuned": sim_finetuned
    })

# ==== Save Results ====
df = pd.DataFrame(results)
df.to_csv("/content/drive/MyDrive/programmingAssignmentShreya/speaker_identity_eval_trial.csv", index=False)

acc_base = df["correct_base"].mean()
acc_finetuned = df["correct_finetuned"].mean()

print(f"\n✅ Rank-1 Accuracy (Pretrained): {acc_base*100:.2f}%")
print(f"✅ Rank-1 Accuracy (Fine-tuned): {acc_finetuned*100:.2f}%")
print("📁 Results saved to: /content/speaker_identity_eval.csv")


🔍 Building speaker embedding bank...
✅ Speaker bank size: 28 speakers
🎙️ Evaluating separated files...


100%|██████████| 20/20 [00:33<00:00,  1.66s/it]


✅ Rank-1 Accuracy (Pretrained): 0.00%
📁 Results saved to: /content/speaker_identity_eval.csv
